In [31]:
#XML Library
import xml.etree.ElementTree as ET
#Natural Language Toolkit
import nltk
from nltk import pos_tag, word_tokenize

In [34]:
#To start I am just using a8.crf to test things out
devFile = "./dev/a8.crf"

In [35]:
#Create XML structure
tree = ET.parse(open(devFile))
tree1 = ET.parse(devFile)
root = tree.getroot()

###To remove all XML tags, it just takes this one line
notags = ET.tostring(root, encoding='utf8', method='text')

In [42]:
#tagged_text is a multidimensional array with each word and its POS tag
tagged_text = nltk.pos_tag(word_tokenize(notags))

In [45]:
#These are all of the NLTK tags that deal with nouns and pronouns
noun_tags = ["NN","NNS","NNPS","PRP","PRP$","WP","WP$"]    

candidate_words = []

#Create a list of words in the text that are nouns/pronouns
for word in tagged_text:
    if word[1] in noun_tags:
        candidate_words.append(word[0])

In [46]:
#Checking to see if candidate_words is populated
print candidate_words

['use', 'clients', 'c.1996', 'number', 'aircraft', 'nation', 'plutonium', 'studies', 'way', 'flight', 'training', 'planes', 'passes', 'flight', 'plant', 'secretary', 'farmers', 'concern', 'plane', 'crash', 'plutonium', 'storage', 'bunker', 'smoke', 'miles', 'Plutonium', 'element', 'cancer', 'risk', 'scenario', 'chance', 'flight', 'data', 'year', 'safety', 'board', 'memo', 'me', 'who', 'land', 'plant', 'miles', 'miles', 'airport', 'We', 'planes', 'time', 'air', 'traffic', 'manager', 'aircraft', 'count', 'count', 'pad', 'air', 'traffic', 'controllers', 'accounting', 'planes', 'day', 'flight', 'patterns', 'zone', 'plutonium', 'bunkers', 'earth', 'grass', 'contractor', 'workers', 'whose', 'mission', 'warheads', 'plutonium', 'ball', 'core', 'weapon', 'weapons', 'assembly', 'plant', 'airport', 'averages', 'takeoffs', 'landings', 'day', 'aircraft', 'planes', 'bases', 'he', 'percent', 'airport', 'traffic', 'flight', 'pattern', 'plutonium', 'storage', 'bunkers', 'months', 'air', 'traffic', 'con

In [43]:
#Print the contents of tagged_text just to see that it is working
print tagged_text

[('nyt960214.0765', 'JJ'), ('A4505', 'NNP'), ('BC-PANTEX-FLIGHTS-TEX', 'NNP'), ('02-14', 'JJ'), ('0535', 'CD'), ('BC-PANTEX-FLIGHTS-TEX', 'NNP'), ('FAA', 'NNP'), ('UNDERESTIMATED', 'NNP'), ('NUMBER', 'NNP'), ('OF', 'NNP'), ('FLIGHTS', 'NNP'), ('OVER', 'NNP'), ('PLUTONIUM', 'NNP'), ('STORAGE', 'NNP'), ('AREA', 'NNP'), ('IN', 'NNP'), ('PANHANDLE', 'NNP'), ('(', '('), ('For', 'IN'), ('use', 'NN'), ('by', 'IN'), ('NYTimes', 'NNP'), ('News', 'NNP'), ('Service', 'NNP'), ('clients', 'NNS'), (')', ')'), ('By', 'IN'), ('HOLLACE', 'NNP'), ('WEINER', 'NNP'), ('c.1996', 'NN'), ('Fort', 'NNP'), ('Worth', 'NNP'), ('Star-Telegram', 'NNP'), ('The', 'DT'), ('Federal', 'NNP'), ('Aviation', 'NNP'), ('Administration', 'NNP'), ('underestimated', 'VBD'), ('the', 'DT'), ('number', 'NN'), ('of', 'IN'), ('aircraft', 'NN'), ('flying', 'VBG'), ('over', 'IN'), ('the', 'DT'), ('Pantex', 'NNP'), ('Weapons', 'NNP'), ('Plant', 'NNP'), ('outside', 'IN'), ('Amarillo', 'NNP'), (',', ','), ('Texas', 'NNP'), (',', ','), (

The slides on coreference contain the following:
Typical Coreference Pipeline
-Preprocessor: XML removal, tokenization, sentence and paragraph splitting
-Part of Speech Tagging
-Parsing
-Named Entity Recognizer
-Semantic Class Lookup (usualy via WordNet)
-Candidate NP extraction
-Supervised Learning
-Clustering into Chains

In [4]:
#Part of Speech Tagging to Identify Nouns, Noun Phrases, and Pronouns

In [5]:
#Create a dictionary and the list of each coreference
#I'm not sure which is more practical or if either is even needed
def createCoref():
    global root
    for child in root:
        coref[str(child.get('ID'))] = child.text
        corefs.append(child.text)

In [12]:
print corefs

['PANTEX', 'FLIGHTS', 'Federal Aviation Administration', 'the number', 'Pantex Weapons Plant', 'plutonium', 'FAA', 'passes', 'flight', 'the plant', 'Energy', 'Plutonium', 'a highly radioactive\nelement', 'one chance in\n10 million', 'flight', 'this year', 'a safety board', 'Trish Neusch', 'Amarillo', 'the airport', 'there', 'these planes', 'the FAA air traffic manager', "a ``manual count on a pad,''", 'That 60-day accounting', '25 planes a day', 'Pantex', 'the Energy Department', 'plutonium', 'the former weapons assembly plant', 'The Amarillo airport', 'McNulty', 'military planes', 'Texas', 'he', 'About 25 percent of the airport traffic', 'a flight pattern\nover the plutonium storage bunkers', 'McNulty', 'air traffic controllers', 'the bunkers', 'he', 'airport', 'the\nnuclear storage area', 'McNulty', 'he', 'VORTAC', 'Amarillo', 'he', 'McNulty', 'these planes', 'the plutonium area', 'Oklahoma', 'the plant', 'McNulty', 'They', '02-14-96']


In [6]:
#Given text and the coref ID, checks to see if any other coref has the exact same text
#If so, sets found to True and sets REF to the ID of the coreference that it matches
def checkExactMatch(coRefText, coID):
    for cr in coref:
        if coref[cr] == coRefText and str(cr) != coID:
            #print "have a match"
            #print "the coRefText is: " + coRefText + " the coID is: " + coID + " the matched is " + cr
            global REF
            global found 
            found = True
            REF = cr

In [20]:
#Checks to see if the coref has potential for an acronym (actually an initialism) 
#and if that acronym is used
def checkAcronym(coRefText):
    #Check if the coref is all caps
    skip = ["and","or","in","of","&"]
    if coRefText.isupper():
        #check list of corefs to see if any of them could match
        for cr in coref:
            words = str(coref[cr])
            acr = ""
            acr2 = ""
            #check if coref is more than one word
            if len(words.split()) > 1:
                #Get the first letter of each word and build candidate acronym
                for word in words.split():
                    if not(word.lower() in skip):
                        acr += word[0]
                    acr2 += word[0]
                #If candidate acronym matches coref, assign the REF
                if acr == coRefText or acr2 == coRefText:
                    global REF
                    global found
                    found = True
                    REF = cr
                    print acr
                    print acr2

In [8]:
#Create Coref variables
coref = {}
corefs = []
createCoref()

In [9]:
#Loop through each coref to assign ref
for co in root.iter('COREF'):
    coID = co.attrib['ID']
    coRefText = co.text
    REF = ''
    found = False
    
    # Find the correct REF
    
    #### Check String Match ####
    # check exact match in other corefs
    checkExactMatch(coRefText, coID)
    if found == True:
        co.set('REF',REF)
        continue 
    
    #check acronyms
    checkAcronym(coRefText)
    if found == True:
        co.set('REF',REF)
        continue 
    # check head noun match
    # check substring and partial match
    # check word overlap comparisons
    
    #### Check Lexical Similarity
    # i.e. Ford Motor Company = Ford Co. = Ford
    # i.e. Apple Computer = Apple
    # i.e. Federabl Bureau of Investigation = FBI
    
    #### Look through candidate list of Nouns checking using
    # scoping heuristics and recency
    # number, person, and gender agreement
    # sytactic heuristics
    # semantic compatability
    
    # Set the REF value
    

NameError: name 'root' is not defined

In [10]:
#Print out the entire XML document
print ET.dump(tree)

<TXT>
 nyt960214.0765 
 A4505 
 BC-PANTEX-FLIGHTS-TEX 
 02-14 
 0535 

BC-<COREF ID="1">PANTEX</COREF>-<COREF ID="2">FLIGHTS</COREF>-TEX
FAA UNDERESTIMATED NUMBER OF FLIGHTS OVER PLUTONIUM STORAGE AREA IN
PANHANDLE
(For use by NYTimes News Service clients)
By HOLLACE WEINER
c.1996 Fort Worth Star-Telegram



The <COREF ID="4">Federal Aviation Administration</COREF> underestimated <COREF ID="5">the number</COREF> of
aircraft flying over the <COREF ID="7">Pantex Weapons Plant</COREF> outside Amarillo,
Texas, where much of the nation's surplus <COREF ID="8" REF="41">plutonium</COREF> is stored,
according to computerized studies under way by the Energy
Department.


The <COREF ID="9" REF="4">FAA</COREF> counted each flight at Amarillo International Airport
once. But military training planes make up to 30 <COREF ID="10">passes</COREF> per <COREF ID="11" REF="21">flight</COREF>
over <COREF ID="12" REF="74">the plant</COREF>, according to the Defense Nuclear Facilities Safety
Board.


The sec

In [22]:
print notags


 nyt960214.0765 
 A4505 
 BC-PANTEX-FLIGHTS-TEX 
 02-14 
 0535 

BC-PANTEX-FLIGHTS-TEX
FAA UNDERESTIMATED NUMBER OF FLIGHTS OVER PLUTONIUM STORAGE AREA IN
PANHANDLE
(For use by NYTimes News Service clients)
By HOLLACE WEINER
c.1996 Fort Worth Star-Telegram



The Federal Aviation Administration underestimated the number of
aircraft flying over the Pantex Weapons Plant outside Amarillo,
Texas, where much of the nation's surplus plutonium is stored,
according to computerized studies under way by the Energy
Department.


The FAA counted each flight at Amarillo International Airport
once. But military training planes make up to 30 passes per flight
over the plant, according to the Defense Nuclear Facilities Safety
Board.


The secretary of Energy and local farmers have expressed concern
that a plane crash into a plutonium storage bunker at Pantex could
spread radioactive smoke for miles. Plutonium, a highly radioactive
element, causes cancer if inhaled.


The risk of that scenario, previo

I thought I may use these but as I think things through, these probably won't be used.

In [ ]:
#The defined functions I've looked at so far

#Adds a REF value for the XML node with a specific ID. Probably won't end up using this.
def setReference(ID,REF):
    for co in root.iter('COREF'):
        coID = co.attrib['ID']
        if coID == ID:
            co.set('REF',REF)


        
        

In [ ]:
for cr in coref:
    print coref[cr]
    
for cr in corefs:
    print cr
    
#Prints out the Corefs just to check the values
for child in root:
    print child.attrib, child.text
    
#Saving the full text of the file to fullText
with open(devFile, 'r') as myfile:
    fullText=myfile.read()